# Project Information

In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../input/support-documentation/support_documentation.pdf


In [73]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -qU langchain-core langchain-text-splitters langchain 
!pip install -qU transformers sentence-transformers chromadb pypdf langchain
!pip install -qU "langchain[cohere]"
!pip install -qU langchain-community
!pip install -qU chromadb
!pip install -qU "langchain-chroma>=0.1.2"
!pip install  --upgrade langchain langchain-core langchain-cohere
!pip uninstall -y ypy-websocket
!pip install langchain-core "langgraph>0.2.27" 
!pip install -Uq "unstructured-client<0.30.0"
!pip uninstall -y aiofiles
!pip install "aiofiles>=24.1.0"
!pip install langchain-community
!pip install pypdf
!pip install "unstructured[pdf]"
!pip install chromadb
%config Completer.use_jedi = False

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.2 requires chromadb!=0.5.10,!=0.5.11,!=0.5.12,!=0.5.4,!=0.5.5,!=0.5.7,!=0.5.9,<0.7.0,>=0.4.0, but you have chromadb 1.0.4 which is incompatible.


In [74]:
from langchain_chroma import Chroma

In [75]:
from kaggle_secrets import UserSecretsClient

CO_API_KEY = UserSecretsClient().get_secret("COHERE_API_KEY")

In [76]:
from langchain.chat_models import init_chat_model



In [77]:
model = init_chat_model("command-r-plus", 
                        model_provider="cohere",
                        cohere_api_key=CO_API_KEY)

In [78]:
response = model.invoke("Tell me a short story about a curious cat")

In [79]:
print(response.content)

Once upon a time, there was a curious cat named Whiskers. He lived in a cozy house with his owner, Mrs. Taylor, who loved him dearly. Whiskers was always curious about the world around him and loved to explore. He would often wander into the garden, chasing after butterflies and birds, always curious about what they were doing and where they were going.

One day, Whiskers noticed a small hole in the fence that separated Mrs. Taylor's garden from the neighbour's yard. He had never been to the other side and was curious about what he might find there. So, he squeezed through the hole and found himself in a whole new world of adventure.

The neighbour's yard was full of interesting smells and sounds, and Whiskers spent hours exploring every corner. He climbed trees, chased after squirrels, and even found a hidden pond where he could cool off on a hot day. But as the sun began to set, Whiskers realized he didn't know how to get back home.

Whiskers meowed loudly, calling out for Mrs. Taylo

Found existing installation: aiofiles 24.1.0
Uninstalling aiofiles-24.1.0:
  Successfully uninstalled aiofiles-24.1.0
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.32.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
google-spark-connect 0.5.2 requires google-

In [81]:
!pip install chromadb

# Define loader, text splitters 
The next step is to get the loader and the text spliters so that we can load
and split the text document to get into a array of Documents

In [82]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


In [83]:
loader = DirectoryLoader(path="..", glob="**/*.pdf", 
                         recursive=True, 
                         show_progress=True)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
docs: Document = loader.load_and_split()

100%|██████████| 1/1 [00:00<00:00,  6.88it/s]


In [84]:
!pip install langchain-chroma


In [85]:
from langchain_cohere.embeddings import CohereEmbeddings
from langchain_cohere.chat_models import ChatCohere

embeddings = CohereEmbeddings(model="embed-english-light-v3.0", cohere_api_key=CO_API_KEY)

In [86]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name = "default_collection",
    embedding_function = embeddings,
    persist_directory="./default_collection_db",
)

retriever = vector_store.as_retriever()

In [87]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]

In [88]:
doc_ids = vector_store.add_documents(uuids = uuids, documents = docs)
doc_ids

['292197b9-5d6a-4f9e-8bfe-77b697e69a31',
 'd0888200-3126-4a17-b689-597f4307b315']

In [89]:
# Test the similarity search 
# vector_store.similarity_search("monitor not working", k=2, )

In [90]:
llm = ChatCohere(model="command-r-plus", cohere_api_key=CO_API_KEY)

In [91]:
from langchain_core.prompts import ChatPromptTemplate
retrieval_prompt = ChatPromptTemplate.from_template(
    """Answer the user's question based on the context provided below:
    
    Context:
    {context}

    If you are unable to answer based on the document then say "I do not have that
    information with me."
    
    Question: {question}
    
    Answer:"""
)

In [92]:
!pip install langgraph

In [93]:
# Let's do the agent now
from typing import TypedDict, Dict
class AgentState(TypedDict):
    keys: Dict[str, any]


from langgraph.graph import StateGraph, END

In [94]:
state:AgentState = {"keys":{}}
user_input = "My mouse is not working, what might be the problem?"
state = {"keys": {"question": user_input}}
retriever = vector_store.as_retriever(embedding=embeddings, k = 2, )
question = state["keys"]["question"]
retrieved_docs = retriever.get_relevant_documents(question)
len(retrieved_docs)

4

In [95]:
workflow = StateGraph(AgentState)

def retrieve_documents(state) -> AgentState:
    """Retrieves relevant documents based on the user's question."""
    question = state["keys"]["question"]
    if question is None:
        raise KeyError(f"No question in state {question}")
    print(f"state is {state}")
    retriever = vector_store.as_retriever(embedding=embeddings, k = 2, )
    retrieved_docs = retriever.get_relevant_documents(question)
    context =  {"context": "\n\n".join([doc.page_content for doc in retrieved_docs])}
    return {"keys":{"question":question, "context":context,}}

def generate_response(state) -> AgentState:
    """Generates the final response using the Cohere model and retrieved context."""
    context = state["keys"]["context"]
    question = state["keys"]["question"]
    # Use the retrieval_prompt here
    response = llm.invoke(retrieval_prompt.format_messages(context=context, question=question))
    state["keys"]["response"] = response
    return state


workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("generate", generate_response)

workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

workflow.set_entry_point("retrieve")

agent = workflow.compile()


In [96]:
state:AgentState = {"keys":{
    "question":"",
    "context":"",
}}
user_input = "My mouse is not working, what might be the problem?"
state = {"keys": {"question": user_input}}
state = agent.invoke(state)

print(state["keys"]["response"])

state is {'keys': {'question': 'My mouse is not working, what might be the problem?'}}
content="The issue of a non-functional mouse could be due to several reasons. Here are some possible causes and troubleshooting steps to identify and resolve the problem:\n\n- **Battery or Power Issues**: Check if the mouse has a low battery or if it's not charged. Replace the batteries or recharge the mouse if necessary. If it's a wired mouse, ensure the power cable is securely connected to both the mouse and the computer.\n\n- **Connection Problems**: Verify that the mouse is properly connected to the computer. If it's a wireless mouse, ensure it's paired or connected to the correct Bluetooth or USB receiver. Try reconnecting or re-pairing the device if necessary.\n\n- **Driver Issues**: Outdated or corrupt mouse drivers can cause the mouse to malfunction. Update your mouse drivers or reinstall them if needed. You can usually do this through your device manager or the manufacturer's support website